In [ ]:
from bs4 import BeautifulSoup
import csv
import requests
from time import sleep
import openpyxl
import random

base_url = "https://www.olx.ba/"
other = "/pretraga?vrsta=samoprodaja&kategorija=23&sort_order=desc&kanton=9&grad%5B0%5D=3812&grad%5B1%5D=3969&grad%5B2%5D=5896&grad%5B3%5D=4048"
links = []


# first run(getting all apartments link)

while other:
    response = requests.get(f"{base_url}{other}", headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")
    stanovi = soup.find_all("div", class_="naslov")

    for stan in stanovi:
        link = stan.find("a", href = True)
        if link != None:
            links.append(link['href'])
            
    print(f"Finished {other}")
    

    try:
        next_page = soup.find("a", rel = "next")["href"]
        other = next_page
    
    except:
        print("Passed through all pages :)")
        break

    sleep(random.randint(2, 6))

    
    
# second run(scrape every apartment for it's info (Header, price, ...))

svi_stanovi = []

for link in links:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")
    
    svi_stanovi.append({

    "naslov": soup.find(id = "naslovartikla").text.strip(),
    "lokacija": soup.find(style = "font-size:14px;background-color:#4d7094;color:#fff;").text.strip(),
    "cijena": soup.find_all("p", style = True)[1].text.strip(),  
    "datum objave": soup.find("time").text,
    "broj kvadrata": soup.find_all("div", class_ = "df2")[0].text,
    "sprat": soup.find_all("div", class_ = "df2")[2].text,
    "broj soba": soup.find_all("div", class_ = "df2")[1].text,  
    "namješten": soup.find_all("div", class_ = "df2")[4].text,
    "vrsta grijanja": soup.find_all("div", class_ = "df2")[3].text,  
    "link":link
        
    })
    
    print(f"Working on {link}")
    sleep(random.randint(2, 6))
    
    
# Now just write all data to csv

filename = "Stanovi.csv"

f = open(filename, "w", encoding="utf-16")

headers = 'naslov;lokacija;cijena;datum objave;broj kvadrata;sprat;broj soba;namješten;vrsta grijanja;link \n' 

f.write(headers)

for stan in svi_stanovi:
    f.write( 
            stan.get("naslov") + ";" +
            stan.get("lokacija") + ";" +
            stan.get("cijena") + ";" +
            stan.get("datum objave") + ";" +
            stan.get("broj kvadrata") + ";" +
            stan.get("sprat") + ";" +
            stan.get("broj soba") + ";" +
            stan.get("namješten") + ";" +
            stan.get("vrsta grijanja") + ";" +
            stan.get("link") + "\n"
            )
f.close()

print("You got your data :D")